In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.impute import SimpleImputer

from scipy.stats import pearsonr
import statsmodels.api as sm

In [ ]:
df_adm = pd.read_csv("/content/drive/Shareddrives/Multicampus/mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz")

In [ ]:
df_dummy = pd.read_csv("/content/drive/Shareddrives/Multicampus/Preprocessing/0414_dummy(최종).csv")

In [ ]:
def prepro(df_adm, df_dummy):  #필요한 전처리 진행 함수
    # 입원 및 퇴원 시간 datetime 형태로 변경
    df_adm['ADMITTIME'] = pd.to_datetime(df_adm['ADMITTIME'])
    df_adm['DISCHTIME'] = pd.to_datetime(df_adm['DISCHTIME'])

    # 체류시간 계산(일단위)
    df_adm['stay'] = (df_adm['DISCHTIME'] - df_adm['ADMITTIME']).dt.total_seconds() / 86400

    # 체류시간 이상치 제거
    df_adm = df_adm[df_adm['stay'] > 0]

    # 쓰지 않는 열 삭제
    df_dummy.drop(['staytime','new_code','AGEG1','AGEG2'], axis=1, inplace=True)

    # 사용할 최종 데이터 추출
    df_clean = pd.merge(df_adm[['SUBJECT_ID','stay']], df_dummy, 
                        on='SUBJECT_ID', how='inner')
    
    return df_clean

In [ ]:
def mean_z(data, target, strategy='mean'):  #데이터 표준화 및 train/test 분리 함수
    content = data.drop(['SUBJECT_ID','HOSPITAL_EXPIRE_FLAG','stay'],axis=1)

    imputer = SimpleImputer(strategy=strategy)
    imputed = imputer.fit_transform(content)
    df_imputed = pd.DataFrame(imputed, columns=content.columns)
        
    X = df_imputed
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 표준화
    sc = StandardScaler()
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    X_test = sc.transform(X_test)

    return X_train, X_test, y_train, y_test

In [ ]:
df_clean = prepro(df_adm, df_dummy)

In [ ]:
df_clean

SUBJECT_ID      stay  AGE  ...  ETH_WHITE  GEN_F  GEN_M
0              22  1.144444   64  ...          1      1      0
1              24  2.856944   39  ...          1      0      1
2              25  3.534028   58  ...          1      0      1
3              26  6.988889   72  ...          0      0      1
4              28  5.364583   74  ...          1      0      1
...           ...       ...  ...  ...        ...    ...    ...
30858       98794  4.145833   78  ...          1      0      1
30859       98797  0.662500   88  ...          1      0      1
30860       98800  2.742361   19  ...          1      1      0
30861       98802  0.548611   83  ...          1      1      0
30862       98805  8.203472   42  ...          1      0      1

[30863 rows x 63 columns]

In [ ]:
X_train, X_test, y_train, y_test = mean_z(df_clean, 'stay')

In [ ]:
X_train

array([[ 0.04850882, -0.61252116,  0.17074561, ..., -1.55750955,
        -0.8729611 ,  0.8729611 ],
       [ 0.88297597, -0.61252116, -0.28673775, ...,  0.64205064,
        -0.8729611 ,  0.8729611 ],
       [-0.1183846 , -0.61252116,  0.62822897, ..., -1.55750955,
        -0.8729611 ,  0.8729611 ],
       ...,
       [ 0.49355797, -0.61252116, -0.74422111, ...,  0.64205064,
         1.14552641, -1.14552641],
       [-1.62042547, -0.61252116, -1.20170446, ...,  0.64205064,
        -0.8729611 ,  0.8729611 ],
       [ 0.43792683, -0.61252116, -0.74422111, ...,  0.64205064,
         1.14552641, -1.14552641]])

In [ ]:
print(X_train.shape[0])
print(X_test.shape[0])

24690
6173


In [ ]:
from sklearn.model_selection import GridSearchCV

tuned_parameters = [{
                     'max_depth' : [2, 3, 4],
                     #'min_samples_split' : [60, 70, 80]
                     }]

# 모델 생성
reg_model = GradientBoostingRegressor(n_estimators=180, random_state=0, min_samples_split=60)  
grid = GridSearchCV(reg_model, tuned_parameters)
grid.fit(X_train, y_train)
reg_model_optimized = grid.best_estimator_

# grid 결과 출력
print(grid.best_score_)
print(grid.best_estimator_)

0.23389008093820998
GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=60,
                          min_weight_fraction_leaf=0.0, n_estimators=180,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=0, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


In [ ]:
y_test_preds = reg_model_optimized.predict(X_test)
r2_optimized = r2_score(y_test, y_test_preds)
print("Optimized R2 score is: {:2f}".format(r2_optimized))

Optimized R2 score is: 0.270927


 loss='ls', max_depth=3, min_samples_split=60, n_estimators=180 => 0.2709
 

In [ ]:
# 모델 생성
gbr_model = GradientBoostingRegressor(random_state=0, n_estimators=180, max_depth=3, min_samples_split=60)
gbr_model.fit(X_train, y_train)
# 예측
y_pred = gbr_model.predict(X_test)
# 결과 확인
r2 = r2_score(y_test, y_pred)
print("Optimized R2 score is: {:2f}".format(r2))

Optimized R2 score is: 0.270927


# 추가 - DNN 모델

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

In [ ]:
model = Sequential()

model.add(Dense(units=35, activation='relu', input_dim=60))
model.add(Dense(units=15, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [ ]:
sgd = optimizers.SGD(lr=0.001)
model.compile(optimizer = sgd ,loss='mse',metrics=['mse'])

In [ ]:
model.fit(X_train, y_train, epochs=500, batch_size=10000, validation_split = 0.2)

Epoch 1/500
2/2 [==============================] - 1s 248ms/step - loss: 202.4281 - mse: 202.4281 - val_loss: 218.2711 - val_mse: 218.2711
Epoch 2/500
2/2 [==============================] - 0s 64ms/step - loss: 197.2031 - mse: 197.2031 - val_loss: 212.6687 - val_mse: 212.6687
Epoch 3/500
2/2 [==============================] - 0s 62ms/step - loss: 192.0288 - mse: 192.0288 - val_loss: 207.2643 - val_mse: 207.2643
Epoch 4/500
2/2 [==============================] - 0s 58ms/step - loss: 181.9952 - mse: 181.9952 - val_loss: 201.7224 - val_mse: 201.7224
Epoch 5/500
2/2 [==============================] - 0s 56ms/step - loss: 183.0231 - mse: 183.0231 - val_loss: 195.8013 - val_mse: 195.8013
Epoch 6/500
2/2 [==============================] - 0s 58ms/step - loss: 173.3595 - mse: 173.3595 - val_loss: 189.3552 - val_mse: 189.3552
Epoch 7/500
2/2 [==============================] - 0s 58ms/step - loss: 167.2600 - mse: 167.2600 - val_loss: 182.2564 - val_mse: 182.2564
Epoch 8/500
2/2 [================

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(r2_score(y_test, y_pred))
print(metrics.mean_absolute_error(y_test, y_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.2542356051378648
5.137993939101363
8.2963465191775


batch_size=10000 & 35/15/8/1 => 0.2318

batch_size=1000 => 0.1926

40/20/10/1 ->  0.2207

30/15/7/1 -> 0.2270

In [ ]:
sgd = optimizers.SGD(lr=0.0001)
model.compile(optimizer = sgd ,loss='mse',metrics=['mse'])

In [ ]:
model.fit(X_train, y_train, epochs=500, batch_size=10000, validation_split = 0.2)

Epoch 1/500
2/2 [==============================] - 1s 223ms/step - loss: 71.9986 - mse: 71.9986 - val_loss: 92.9710 - val_mse: 92.9710
Epoch 2/500
2/2 [==============================] - 0s 61ms/step - loss: 71.9382 - mse: 71.9382 - val_loss: 92.9704 - val_mse: 92.9704
Epoch 3/500
2/2 [==============================] - 0s 61ms/step - loss: 73.9993 - mse: 73.9993 - val_loss: 92.9700 - val_mse: 92.9700
Epoch 4/500
2/2 [==============================] - 0s 59ms/step - loss: 71.9408 - mse: 71.9408 - val_loss: 92.9696 - val_mse: 92.9696
Epoch 5/500
2/2 [==============================] - 0s 65ms/step - loss: 72.6364 - mse: 72.6364 - val_loss: 92.9693 - val_mse: 92.9693
Epoch 6/500
2/2 [==============================] - 0s 64ms/step - loss: 72.2015 - mse: 72.2015 - val_loss: 92.9689 - val_mse: 92.9689
Epoch 7/500
2/2 [==============================] - 0s 62ms/step - loss: 71.2237 - mse: 71.2237 - val_loss: 92.9687 - val_mse: 92.9687
Epoch 8/500
2/2 [==============================] - 0s 60ms/st

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(r2_score(y_test, y_pred))
print(metrics.mean_absolute_error(y_test, y_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.25546302008185373
5.133022914721006
8.289516444449903


# Cross Variation

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense 
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score 

In [ ]:
# fix random seed for reproducibility 
seed = 7 
np.random.seed(seed)

# create model 
model = GradientBoostingRegressor(random_state=0, n_estimators=180, max_depth=3, min_samples_split=60)   

kfold = KFold(n_splits=10, shuffle=True, random_state=seed) 
results = cross_val_score(model, X_train, y_train, cv=kfold)

In [ ]:
print(results)
print(results.mean())

[0.27973051 0.24472196 0.23192026 0.23597576 0.24735544 0.208401
 0.2281828  0.25176122 0.15774336 0.1997173 ]
0.2285509595337095
